In [19]:
import requests
import json
import time
import pandas as pd
from tqdm import tqdm

# full list of spheres may be found here: https://api.hh.ru/specializations
def get_specializations(sphere_id = 1): 
    response = requests.get('https://api.hh.ru/specializations')
    js_obj = json.loads(response.text)
    response.close()
    
    for sphere in js_obj:
        if sphere['id'] == str(sphere_id):
            specializations = sphere['specializations']
            break
            
    ids = dict()
    for spec in specializations:
        ids[spec['id']] = spec['name']
        
    return ids

def get_page(page = 0, specialization = None):
    params = {
        "specialization": "1.110",
        'area': 1, 
        'page': page, 
        'per_page': 100}
    req = requests.get('https://api.hh.ru/vacancies', params) 
    data = req.content.decode() 
    req.close()
    return data
        
def get_vacancy_data(vacancy):
    _id = vacancy['id']
    _name = vacancy['name']
    _date = vacancy['published_at']
    
    ### Salary ###
    try:
        _salary_from = vacancy['salary']['from']
    except:
        _salary_from = None
    try:
        _salary_to = vacancy['salary']['to']
    except:
        _salary_to = None
    try:
        _cur = vacancy['salary']['currency']
    except:
        _cur = None
    try:
        _gross = vacancy['salary']['gross']
    except:
        _gross = None
        
    ### Employer ###
    try:
        _employer_id =  vacancy['employer']['id']
        _employer_name =  vacancy['employer']['name']
    except:
        _employer_id, _employer_name = None, None
        
    ### Requirements & Responsibilities ###
    try:
        _requirements = vacancy['snippet']['requirement']
        _responsibility = vacancy['snippet']['responsibility']
    except:
        _requirements, _responsibility = None, None
        
    ### Schedule ###
    try:
        _schedule = vacancy['schedule']['name']
    except:
        _schedule = None
        
    ### Location ###
    try:
        _city = vacancy['address']['city']
    except:
        _city = None
        
    try:
        _address = vacancy['address']['raw']
    except:
        _address = None
        
    try: 
        _metro = vacancy['address']['metro']['station_name']
        _line = vacancy['address']['metro']['line_name']
    except:
        _metro, _line = None, None
        
    data = {'id' : _id, 'name' : _name, 'date' : _date,
            'salary_from' : _salary_from, 'salary_to': _salary_to,
            'cur' : _cur, 'gross' : _gross, 
            'employer_id' : _employer_id, 'employer_name' : _employer_name,
            'requirements' : _requirements, 'responsibility' : _responsibility,
            'schedule' : _schedule, 'city' : _city, 'address' : _address,
            'metro' : _metro, 'metro_line' : _line}
    
    return data
    

In [21]:
ids = get_specializations(sphere_id = 1)
df = pd.DataFrame()
# Считываем первые 2000 вакансий
for specialisation in tqdm(list(ids.keys())):
    for page in range(0, 20):
        data = get_page(page = page, specialization = specialisation)
        js_obj = json.loads(data)
        
        for vacc in js_obj['items']:
            vacc_data = get_vacancy_data(vacc)
            vacc_data['profearea_id'] = specialisation
            vacc_data['profearea_name'] = ids[specialisation]
            vacc_frame = pd.DataFrame.from_dict(vacc_data, orient = 'index').T
            df = df.append(vacc_frame)
            
        if (js_obj['pages'] - page) <= 1:
            break
        time.sleep(0.5)
        
df = df.reset_index(drop = True)
df

100%|███████████████████████████████████████████| 37/37 [18:27<00:00, 29.92s/it]


,id,name,date,salary_from,salary_to,cur,gross,employer_id,employer_name,requirements,responsibility,schedule,city,address,metro,metro_line,profearea_id,profearea_name
0,50361418,Риск-менеджер (переезд в Турцию),2021-12-10T11:57:42+0300,None,None,None,None,2347,TUI Russia & CIS,None,"Лидировать процесс предупреждения, выявления, ...",Полный день,Москва,"Москва, улица Викторенко, 5с1",Аэропорт,Замоскворецкая,1.395,Банковское ПО
1,50410075,Специалист по управлению рисками экономической...,2022-01-11T11:14:25+0300,None,None,None,None,3982116,СберМаркетинг,Юридическое или Экономическое образование. Опы...,Оценка рисков (благонадежности) в отношении но...,Полный день,Москва,"Москва, Поклонная улица, 3к4",Кутузовская,Филевская,1.395,Банковское ПО
2,49295970,Старший инженер (Cyberark),2021-12-27T13:30:03+0300,None,None,None,None,3415,ИНФОРМЗАЩИТА,Высшее образование (в области ИТ или ИБ). Опыт...,"Проработка технических решений, стендовое моде...",Полный день,None,None,None,None,1.395,Банковское ПО
3,50676085,Начальник отдела системного администрирования ...,2022-01-10T10:12:16+0300,195000,None,RUR,False,4816,Т.Б.М.,Высшее образование. Опыт работы не менее 3-х л...,Анализ текущей ситуации и потребностей компани...,Полный день,None,None,None,None,1.395,Банковское ПО
4,50883352,Специалист по экономической безопасности,2022-01-14T17:35:16+0300,70000,None,RUR,False,104309,Мясницкий ряд,Образование не ниже среднего экономического. З...,Осуществление плановых и внеплановых ревизий х...,Полный день,None,None,None,None,1.395,Банковское ПО
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60909,50887843,Senior Database Administrator,2022-01-10T17:16:25+0300,None,None,None,None,672796,SOFTSWISS,3 years of experience in database administrati...,2009 – year founded. 800 people at the company...,Гибкий график,None,None,None,None,1.359,Электронная коммерция
60910,48505376,Security engineer,2022-01-14T09:40:35+0300,None,None,None,None,3719391,Хайв Экспо Интернешнл,You have worked as a Security engineer for at ...,"Ensure compliance with applicable Policies, St...",Полный день,Москва,"Москва, Верхняя Красносельская улица, 3с2",Красносельская,Сокольническая,1.359,Электронная коммерция
60911,50297094,Imunify Product Marketing Manager [Remote],2022-01-13T09:42:00+0300,None,None,None,None,1245452,Cloud Linux,Technical background and knowledge of server s...,"Work with product, sales engineering, and supp...",Удаленная работа,None,None,None,None,1.359,Электронная коммерция
60912,50735640,Build Engineer,2021-12-28T16:02:43+0300,None,None,None,None,2406247,Selecty,Знание Python. Опыт построения CI/CD. Знание A...,Доработка существующих и/или разработка новых ...,Полный день,None,None,None,None,1.359,Электронная коммерция


In [22]:
df.to_csv('vaccancies.csv', index = False)

In [23]:
df.columns

Index(['id', 'name', 'date', 'salary_from', 'salary_to', 'cur', 'gross',
       'employer_id', 'employer_name', 'requirements', 'responsibility',
       'schedule', 'city', 'address', 'metro', 'metro_line', 'profearea_id',
       'profearea_name'],
      dtype='object')